<a href="https://colab.research.google.com/github/mounesi/pa/blob/master/Distress_Detection_generate_TFRECORD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
from __future__ import division, print_function, absolute_import

import os
from google.colab import drive
import tensorflow.compat.v1 as tf

import urllib.request
import tarfile
import numpy as np

if np.__version__ != '1.17.0':
  ! pip install numpy==1.17
  os.kill(os.getpid(), 9)

# check the versions
print(f'np version is {np.__version__} shall match with 1.17.0')
print(f'tf version is {tf.__version__} shall match with 1.15.2')

TensorFlow 1.x selected.
np version is 1.17.0 shall match with 1.17.0
tf version is 1.15.2 shall match with 1.15.2


In [0]:
# imports 

from PIL import Image
import os
import io
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import csv
import cv2
import pandas as pd

In [3]:
# MODEL DIRECTORY
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -qq Cython contextlib2 pillow lxml matplotlib

!pip install -qq pycocotools

# Downlaods Tenorflow
!git clone --q https://github.com/tensorflow/models.git

Selecting previously unselected package python-bs4.
(Reading database ... 144467 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

In [4]:
cd models/research

/content/models/research


In [5]:
# exports the PYTHONPATH environment variable with the reasearch and slim folders' paths
!protoc object_detection/protos/*.proto --python_out=.
#os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [6]:
pwd

'/content/models/research'

In [0]:
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
#os.environ['PYTHONPATH'] += ':./:./slim/'

In [8]:
!pip install tf_slim

     |████████████████████████████████| 358kB 2.8MB/s 


In [0]:
!python object_detection/builders/model_builder_test.py

In [10]:
pwd

'/content/models/research'

In [0]:
from object_detection.utils import dataset_util

# Data Pre-Processing to Generate TF Record File:


## Organizing Image Dataset and Associated Labels

1. The images are collected from the highway in New York 
2. The images are labeled using LabelImg [source](https://github.com/SkalskiP/make-sense)  
3. make a directory of the project
4. splitting the data for train and test separately.

## Downloading the Data and lables

In [20]:
cd /content

/content


In [0]:
cp drive/My\ Drive/data/pa/labels.zip .

In [0]:
cp drive/My\ Drive/data/pa/street_dataset.zip .

In [0]:
# create subdirectories for test and train
!mkdir data
!mkdir data/images data/train_labels data/test_labels

In [24]:
pwd

'/content'

In [25]:
!unzip street_dataset.zip
!mv street_dataset/* data/images/

Archive:  street_dataset.zip
   creating: street_dataset/
  inflating: street_dataset/image8.jpg  
  inflating: street_dataset/image9.jpg  
  inflating: street_dataset/image14.jpg  
  inflating: street_dataset/image28.jpg  
  inflating: street_dataset/image29.jpg  
  inflating: street_dataset/image15.jpg  
  inflating: street_dataset/image17.jpg  
  inflating: street_dataset/image16.jpg  
  inflating: street_dataset/image12.jpg  
  inflating: street_dataset/image13.jpg  
  inflating: street_dataset/image39.jpg  
  inflating: street_dataset/image11.jpg  
  inflating: street_dataset/image10.jpg  
  inflating: street_dataset/image38.jpg  
  inflating: street_dataset/image21.jpg  
  inflating: street_dataset/image35.jpg  
  inflating: street_dataset/image34.jpg  
  inflating: street_dataset/image20.jpg  
  inflating: street_dataset/image36.jpg  
  inflating: street_dataset/image22.jpg  
  inflating: street_dataset/image23.jpg  
  inflating: street_dataset/image37.jpg  
  inflating: street_

In [26]:
!unzip labels.zip
!mv labels/* data/train_labels

Archive:  labels.zip
   creating: labels/
  inflating: __MACOSX/._labels       
  inflating: labels/image8.xml       
  inflating: __MACOSX/labels/._image8.xml  
  inflating: labels/image9.xml       
  inflating: __MACOSX/labels/._image9.xml  
  inflating: labels/image28.xml      
  inflating: __MACOSX/labels/._image28.xml  
  inflating: labels/image14.xml      
  inflating: __MACOSX/labels/._image14.xml  
  inflating: labels/image15.xml      
  inflating: __MACOSX/labels/._image15.xml  
  inflating: labels/image29.xml      
  inflating: __MACOSX/labels/._image29.xml  
  inflating: labels/image16.xml      
  inflating: __MACOSX/labels/._image16.xml  
  inflating: labels/image12.xml      
  inflating: __MACOSX/labels/._image12.xml  
  inflating: labels/image11.xml      
  inflating: __MACOSX/labels/._image11.xml  
  inflating: labels/image39.xml      
  inflating: __MACOSX/labels/._image39.xml  
  inflating: labels/image38.xml      
  inflating: __MACOSX/labels/._image38.xml  
  inflati

In [27]:
ls data/train_labels/

image0.xml   image18.xml  image24.xml  image30.xml  image37.xml  image7.xml
image10.xml  image19.xml  image25.xml  image31.xml  image38.xml  image8.xml
image11.xml  image1.xml   image26.xml  image32.xml  image39.xml  image9.xml
image12.xml  image20.xml  image27.xml  image33.xml  image3.xml
image14.xml  image21.xml  image28.xml  image34.xml  image4.xml
image15.xml  image22.xml  image29.xml  image35.xml  image5.xml
image16.xml  image23.xml  image2.xml   image36.xml  image6.xml


In [0]:
# remove the zip files and extracted files
!rm -rf labels.zip labels/ street_dataset.zip street_dataset/

In [29]:
#  38 all training labels
!ls -1 data/train_labels/ | wc -l

38


In [0]:
# move 20 percent to the test dataset randomly
!ls data/train_labels/* | sort -R | head -8 | xargs -I{} mv {} data/test_labels

In [31]:
# 30 "images"(xml) for training
!ls -1 data/train_labels/ | wc -l

30


In [32]:
# 8 "images"(xml) for testing
!ls -1 data/test_labels/ | wc -l

8


## Preprocessing Images and Labels
1. Converting the annotations from xml files to two csv files for each `train_labels/` and `train_labels/`.
2. Creating a pbtxt file that specifies the number of class (one class in this case)
3. Checking if the annotations for each object are placed within the range of the image width and height.

In [33]:
cd data

/content/data




```
item {
    id: 1
    name: 'distress'
}

```

In [34]:
# ONCE ONCE ONCE
#adjusted from: https://github.com/datitran/raccoon_dataset
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    #print(xml_file)
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
#      value = (root.find('filename').text + '.jpg',
      value = (root.find('filename').text,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.


In [35]:
#checking the pbtxt file
!cat label_map.pbtxt

item {
    id: 1
    name: 'distress'
}

In [36]:
# they are there!
!ls -l

total 48
drwxr-xr-x 2 root root  4096 Jun  6 09:19 images
-rw-r--r-- 1 root root    39 Jun  6 09:19 label_map.pbtxt
drwxr-xr-x 2 root root  4096 Jun  6 09:19 test_labels
-rw-r--r-- 1 root root  8907 Jun  6 09:19 test_labels.csv
drwxr-xr-x 2 root root  4096 Jun  6 09:19 train_labels
-rw-r--r-- 1 root root 18830 Jun  6 09:19 train_labels.csv


## Catch Errors in the Data

In [37]:
#checks if the images box position is placed within the image.

#note: while this doesn't checks if the boxes/annotatoins are correctly
# placed around the object, Tensorflow will through an error if this occured.
#%cd /content/{project}}/data
# path to images
images_path = 'images'

#loops over both train_labels and test_labels csv files to do the check
# returns the image name where an error is found 
# return the incorrect attributes; xmin, ymin, xmax, ymax.
for CSV_FILE in ['train_labels.csv', 'test_labels.csv']:
  with open(CSV_FILE, 'r') as fid:  
      print('[*] Checking file:', CSV_FILE) 
      file = csv.reader(fid, delimiter=',')
      first = True 
      cnt = 0
      error_cnt = 0
      error = False
      for row in file:
          if error == True:
              error_cnt += 1
              error = False         
          if first == True:
              first = False
              continue     
          cnt += 1      
          name, width, height, xmin, ymin, xmax, ymax = row[0], int(row[1]), int(row[2]), int(row[4]), int(row[5]), int(row[6]), int(row[7])     
          path = os.path.join(images_path, name)
          print(image_path)
          print(name)
          img = cv2.imread(path)         
          if type(img) == type(None):
              error = True
              print('Could not read image', img)
              continue     
          org_height, org_width = img.shape[:2]     
          if org_width != width:
              error = True
              print('Width mismatch for image: ', name, width, '!=', org_width)     
          if org_height != height:
              error = True
              print('Height mismatch for image: ', name, height, '!=', org_height) 
          if xmin > org_width:
              error = True
              print('XMIN > org_width for file', name)  
          if xmax > org_width:
              error = True
              print('XMAX > org_width for file', name)
          if ymin > org_height:
              error = True
              print('YMIN > org_height for file', name)
          if ymax > org_height:
              error = True
              print('YMAX > org_height for file', name)
          if error == True:
              print('Error for file: %s' % name)
              print()
      print()
      print('Checked %d files and realized %d errors' % (cnt, error_cnt))
      print("-----")

[*] Checking file: train_labels.csv
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image4.jpg
/content/data/test_labels
image18.jpg
/content/data/test_labels
image18.jpg
/content/data/test_labels
image26.jpg
/content/data/test_labels
image26.jpg
/content/data/test_labels
image26.jpg
/content/data/test_labels
image26.jpg
/content/data/test_labels
image26

## Fix Data if Any Error is Found

In [0]:
# if removing needed:
#rm images/'image(39).jpg'

#removing the entry for it in the csv for that image as well

#because we did a random split for the data, we dont know if it ended up being in training or testing
# we will remove the image from both.

#training
#reading the training csv
#df = pd.read_csv('/content/{project}}/data/train_labels.csv')
# removing image(39).jpg
#df = df[df['filename'] != 'image(39).jpg']
#reseting the index
#df.reset_index(drop=True, inplace=True)
#saving the df
#df.to_csv('/content/{project}/data/train_labels.csv')


#testing
#reading the testing csv
#df = pd.read_csv('/content/{project}/data/test_labels.csv')
# removing image(39).jpg
#df = df[df['filename'] != 'image(39)'.jpg']
#reseting the index
#df.reset_index(drop=True, inplace=True)
#saving the df
#df.to_csv('/content/{project}/data/test_labels.csv')

# Just for the memory
#df = None


In [39]:
cd ..

/content


## Generating Tf record
- Generating two TFRecords files for the training and testing CSVs.
- Tensorflow accepts the data as tfrecords which is a binary file that run fast with low memory usage. Instead of loading the full data into memory, Tenorflow breaks the data into batches using these TFRecords automatically

In [40]:
cd /content/models/research

/content/models/research


In [0]:
from object_detection.utils import dataset_util
from collections import namedtuple

In [42]:
cd ..

/content/models


In [0]:
DATA_BASE_PATH = "/content/data/"
image_dir = DATA_BASE_PATH +'images/'

In [44]:
image_dir

'/content/data/images/'

In [49]:
#adjusted from: https://github.com/datitran/raccoon_dataset

# converts the csv files for training and testing data to two TFRecords files.
# places the output in the same directory as the input
#from object_detection.utils import dataset_util
#%cd /content/gun_detection/models/

#### THIS MUST BE CHANGED FOR DIFFERENT APPLICATION

def class_text_to_int(row_label):
    if (row_label == 'distress'):
		    return 1
    else:
		    None


def split(df, group):
		data = namedtuple('data', ['filename', 'object'])
		gb = df.groupby(group)
		return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
		with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
				encoded_jpg = fid.read()
		encoded_jpg_io = io.BytesIO(encoded_jpg)
		image = Image.open(encoded_jpg_io)
		width, height = image.size

		filename = group.filename.encode('utf8')
		image_format = b'jpg'
		xmins = []
		xmaxs = []
		ymins = []
		ymaxs = []
		classes_text = []
		classes = []

		for index, row in group.object.iterrows():
				xmins.append(row['xmin'] / width)
				xmaxs.append(row['xmax'] / width)
				ymins.append(row['ymin'] / height)
				ymaxs.append(row['ymax'] / height)
				classes_text.append(row['class'].encode('utf8'))
				classes.append(class_text_to_int(row['class']))

		tf_example = tf.train.Example(features=tf.train.Features(feature={
				'image/height': dataset_util.int64_feature(height),
				'image/width': dataset_util.int64_feature(width),
				'image/filename': dataset_util.bytes_feature(filename),
				'image/source_id': dataset_util.bytes_feature(filename),
				'image/encoded': dataset_util.bytes_feature(encoded_jpg),
				'image/format': dataset_util.bytes_feature(image_format),
				'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
				'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
				'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
				'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
				'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
				'image/object/class/label': dataset_util.int64_list_feature(classes),
		}))
		return tf_example

for csv in ['train_labels', 'test_labels']:
  writer = tf.io.TFRecordWriter(DATA_BASE_PATH + csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(DATA_BASE_PATH + csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
      tf_example = create_tf_example(group, path)
      writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), DATA_BASE_PATH + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(DATA_BASE_PATH +csv + '.record'))

Successfully created the TFRecords: /content/data/train_labels.record
Successfully created the TFRecords: /content/data/test_labels.record
